In [1]:
import zipfile
import os
import pandas as pd


zip_file_path = os.path.join(os.getcwd(), "..", "data", "raw", "airports-database.zip")
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    with zip_ref.open("airports-database.csv") as csv_file:
        df = pd.read_csv(csv_file)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count, date_format, desc, max, month, row_number
from pyspark.sql.window import Window


# inicializando a SparkSession
spark = SparkSession.builder.appName("EDA").getOrCreate()

# convertendo o DataFrame Pandas para um DataFrame Spark
df = spark.createDataFrame(df)

your 131072x1 screen size is bogus. expect trouble
24/10/05 10:16:18 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 172.24.246.235 instead (on interface eth0)
24/10/05 10:16:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/05 10:16:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/05 10:16:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/plbalmeida/picpay-case-machine-learning-engineer/venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is 

In [3]:
# primeiras linhas do DataFrame
df.show()

24/10/05 10:16:56 WARN TaskSetManager: Stage 0 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
| id|year|month|day|dep_time|sched_dep_time|dep_delay|arr_time|sched_arr_time|arr_delay|carrier|flight|tailnum|origin|dest|air_time|distance|hour|minute|          time_hour|                name|
+---+----+-----+---+--------+--------------+---------+--------+--------------+---------+-------+------+-------+------+----+--------+--------+----+------+-------------------+--------------------+
|  0|2013|    1|  1|   517.0|           515|      2.0|   830.0|           819|     11.0|     UA|  1545| N14228|   EWR| IAH|   227.0|    1400|   5|    15|2013-01-01 05:00:00|United Air Lines ...|
|  1|2013|    1|  1|   533.0|           529|      4.0|   850.0|           830|     20.0|     UA|  1714| N24211|   LGA| IAH|   227.0|    1416|   5|    29|2013-01-01 05:00:00|United Air Lines ...|
|  2|2013|    1|  1|   54

In [4]:
# schema da tabela
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- year: long (nullable = true)
 |-- month: long (nullable = true)
 |-- day: long (nullable = true)
 |-- dep_time: double (nullable = true)
 |-- sched_dep_time: long (nullable = true)
 |-- dep_delay: double (nullable = true)
 |-- arr_time: double (nullable = true)
 |-- sched_arr_time: long (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: long (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: double (nullable = true)
 |-- distance: long (nullable = true)
 |-- hour: long (nullable = true)
 |-- minute: long (nullable = true)
 |-- time_hour: string (nullable = true)
 |-- name: string (nullable = true)



In [5]:
# 1. número total de voos
total_flights = df.count()
print(f"Número total de voos: {total_flights}")

24/10/05 10:16:58 WARN TaskSetManager: Stage 1 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


Número total de voos: 336776


In [6]:
# 2. número de voos cancelados (dep_time e arr_time nulos)
cancelled_flights = df.filter(col("dep_time").isNull() & col("arr_time").isNull()).count()
print(f"Número de voos cancelados: {cancelled_flights}")

24/10/05 10:17:00 WARN TaskSetManager: Stage 4 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


Número de voos cancelados: 0


In [7]:
# 3. atraso médio na partida (dep_delay)
avg_dep_delay = df.filter(col("dep_delay").isNotNull() & (col("dep_delay") > 0)).na.drop().select(avg("dep_delay")).collect()[0][0]
print(f"Atraso médio na partida: {avg_dep_delay} minutos")

24/10/05 10:17:01 WARN TaskSetManager: Stage 7 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


Atraso médio na partida: 39.237473090923324 minutos


In [8]:
# 4. 5 aeroportos com maior número de pousos
top5_airports_landings = df.groupBy("dest").agg(count("id").alias("num_landings")).orderBy(desc("num_landings")).limit(5)
top5_airports_landings.show()

24/10/05 10:17:03 WARN TaskSetManager: Stage 10 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+----+------------+
|dest|num_landings|
+----+------------+
| ORD|       17283|
| ATL|       17215|
| LAX|       16174|
| BOS|       15508|
| MCO|       14082|
+----+------------+



In [9]:
# 5. rota mais frequente (par origin-dest)
most_frequent_route = df.groupBy("origin", "dest").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(1)
most_frequent_route.show()

24/10/05 10:17:04 WARN TaskSetManager: Stage 13 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   JFK| LAX|      11262|
+------+----+-----------+



In [10]:
# 6. 5 companhias aéreas com maior tempo médio de atraso na chegada
top5_carriers_avg_arr_delay = df.na.drop().groupBy("carrier").agg(avg("arr_delay").alias("avg_arr_delay")).orderBy(desc("avg_arr_delay")).limit(5)
top5_carriers_avg_arr_delay.show()

24/10/05 10:17:06 WARN TaskSetManager: Stage 16 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------------+
|carrier|     avg_arr_delay|
+-------+------------------+
|     F9|21.920704845814978|
|     FL|20.115905511811025|
|     EV| 15.79643108710965|
|     YV|15.556985294117647|
|     OO|11.931034482758621|
+-------+------------------+



In [11]:
# 7. dia da semana com maior número de voos
day_of_week_with_most_flights = df.withColumn("day_of_week", date_format("time_hour", "E")).groupBy("day_of_week").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(1)
day_of_week_with_most_flights.show()

24/10/05 10:17:07 WARN TaskSetManager: Stage 19 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-----------+
|day_of_week|num_flights|
+-----------+-----------+
|        Mon|      50690|
+-----------+-----------+



In [12]:
# 8. percentual mensal dos voos que tiveram atraso na partida superior a 30 minutos
monthly_delays_over_30 = df.filter(col("dep_delay") > 30).groupBy(month("time_hour").alias("month")).agg((count("id") / df.count() * 100).alias("percent_delays_over_30"))
monthly_delays_over_30.orderBy(col("month")).show()

24/10/05 10:17:09 WARN TaskSetManager: Stage 22 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.
24/10/05 10:17:10 WARN TaskSetManager: Stage 25 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+-----+----------------------+
|month|percent_delays_over_30|
+-----+----------------------+
|    1|      1.14942870038245|
|    2|     1.319274532627028|
|    3|     1.535145022210609|
|    4|    1.5437560871320997|
|    5|    1.4784307670380312|
|    6|     1.997173195239566|
|    7|    2.1120863719504954|
|    8|    1.4027127823835428|
|    9|    0.8524954272275934|
|   10|    0.8709052901631944|
|   11|    0.7782621089388793|
|   12|    1.7507185785210346|
+-----+----------------------+



In [13]:
# 9. origem mais comum para voos que pousaram em Seattle (SEA)
most_common_origin_to_sea = df.filter(col("dest") == "SEA").groupBy("origin").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(1)
most_common_origin_to_sea.show()

24/10/05 10:17:11 WARN TaskSetManager: Stage 28 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+------+-----------+
|origin|num_flights|
+------+-----------+
|   JFK|       2092|
+------+-----------+



In [14]:
# 10. média de atraso na partida dos voos para cada dia da semana
avg_dep_delay_per_day = df.na.drop() \
    .withColumn("day_of_week", date_format("time_hour", "E")) \
    .groupBy("day_of_week") \
    .agg(avg("dep_delay").alias("avg_dep_delay"))

avg_dep_delay_per_day.show()


24/10/05 10:17:12 WARN TaskSetManager: Stage 31 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+-----------+------------------+
|day_of_week|     avg_dep_delay|
+-----------+------------------+
|        Sun|11.477475497736561|
|        Mon|14.718727814851626|
|        Thu| 16.04345133656724|
|        Sat|7.5944065195533685|
|        Wed|11.643321269945714|
|        Tue|10.588355007428211|
|        Fri| 14.65397374873792|
+-----------+------------------+



In [15]:
# 11. rota com maior tempo de voo médio
route_with_max_avg_airtime = df.na.drop().groupBy("origin", "dest").agg(avg("air_time").alias("avg_air_time")).orderBy(desc("avg_air_time")).limit(1)
route_with_max_avg_airtime.show()

24/10/05 10:17:14 WARN TaskSetManager: Stage 34 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+------+----+-----------------+
|origin|dest|     avg_air_time|
+------+----+-----------------+
|   JFK| HNL|623.0877192982456|
+------+----+-----------------+



In [16]:
# 12. aeroporto de destino mais comum para cada origem
window_spec = Window.partitionBy("origin").orderBy(desc("num_flights"))
dest_most_common_per_origin = df.groupBy("origin", "dest").agg(count("id").alias("num_flights"))\
    .withColumn("rank", row_number().over(window_spec))\
    .filter(col("rank") == 1).drop("rank")

dest_most_common_per_origin.show()

24/10/05 10:17:16 WARN TaskSetManager: Stage 37 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   EWR| ORD|       6100|
|   JFK| LAX|      11262|
|   LGA| ATL|      10263|
+------+----+-----------+



In [17]:
# 13. 3 rotas com maior variação no tempo médio de voo
routes_with_max_variation_airtime = df.na.drop().groupBy("origin", "dest").agg((max("air_time") - avg("air_time")).alias("air_time_variation")).orderBy(desc("air_time_variation")).limit(3)
routes_with_max_variation_airtime.show()

24/10/05 10:17:17 WARN TaskSetManager: Stage 43 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+------+----+------------------+
|origin|dest|air_time_variation|
+------+----+------------------+
|   JFK| SFO|142.59637439881612|
|   JFK| EGE|125.55445544554453|
|   JFK| LAX| 110.8489111927592|
+------+----+------------------+



In [18]:
# 14. média de atraso na chegada para voos com atraso na partida superior a 1 hora
avg_arr_delay_for_dep_delays_over_1hr = df.filter(col("dep_delay") > 60).na.drop().select(avg("arr_delay")).collect()[0][0]
print(f"Média de atraso na chegada para voos com atraso na partida superior a 1 hora: {avg_arr_delay_for_dep_delays_over_1hr}")

24/10/05 10:17:19 WARN TaskSetManager: Stage 46 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


Média de atraso na chegada para voos com atraso na partida superior a 1 hora: 119.04880549963919


In [19]:
# 15. média de voos diários para cada mês
daily_avg_flights_per_month = df.withColumn("month", month("time_hour")).groupBy("month").agg((count("id") / 30).alias("avg_daily_flights"))
daily_avg_flights_per_month.orderBy(col("month")).show()

24/10/05 10:17:20 WARN TaskSetManager: Stage 49 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+-----+-----------------+
|month|avg_daily_flights|
+-----+-----------------+
|    1|900.1333333333333|
|    2|            831.7|
|    3|961.1333333333333|
|    4|944.3333333333334|
|    5|959.8666666666667|
|    6|941.4333333333333|
|    7|980.8333333333334|
|    8|977.5666666666667|
|    9|919.1333333333333|
|   10|962.9666666666667|
|   11|908.9333333333333|
|   12|937.8333333333334|
+-----+-----------------+



In [20]:
# 16. 3 rotas mais comuns com atrasos na chegada superiores a 30 minutos
top3_routes_with_arr_delays_over_30 = df.filter(col("arr_delay") > 30).groupBy("origin", "dest").agg(count("id").alias("num_flights")).orderBy(desc("num_flights")).limit(3)
top3_routes_with_arr_delays_over_30.show()

24/10/05 10:17:22 WARN TaskSetManager: Stage 52 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   LGA| ATL|       1785|
|   LGA| ORD|       1538|
|   JFK| LAX|       1389|
+------+----+-----------+



In [21]:
# 17. principal destino para cada origem
window_spec = Window.partitionBy("origin").orderBy(desc("num_flights"))
main_dest_per_origin = df.groupBy("origin", "dest").agg(count("id").alias("num_flights"))\
    .withColumn("rank", row_number().over(window_spec))\
    .filter(col("rank") == 1).drop("rank")

main_dest_per_origin.show()

24/10/05 10:17:27 WARN TaskSetManager: Stage 55 contains a task of very large size (3929 KiB). The maximum recommended task size is 1000 KiB.


+------+----+-----------+
|origin|dest|num_flights|
+------+----+-----------+
|   EWR| ORD|       6100|
|   JFK| LAX|      11262|
|   LGA| ATL|      10263|
+------+----+-----------+



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 33178)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/plbalmeida/picpay-case-machine-learning-engineer/venv/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/plbalmeida/picpay-case-machine-learning-engineer/venv/lib/python3.10/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/plbalmeida/picpa